In [1]:
!git clone https://github.com/clementgr/lymphocytosis-mil.git

Cloning into 'lymphocytosis-mil'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 123 (delta 58), reused 91 (delta 29), pack-reused 0
Receiving objects: 100% (123/123), 20.76 KiB | 6.92 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
import os
os.chdir('lymphocytosis-mil')

In [3]:
%%capture
!pip install -r requirements.txt
!rm -r /content/sample_data/

In [4]:
%%capture
!sh scripts/data_download.sh

train

In [5]:
import torch.nn as nn
from pytorch_lightning import seed_everything, Trainer

from callbacks import ProgressBar
from data.data_module import LymphoDataModule
from models.mil_model import MILModel
from models.simple_cnn import SimpleCNN
from models.resnet import se_resnet50

In [6]:
seed_everything(21)

data_module = LymphoDataModule(
    data_dir='data/3md3070-dlmi/',
    batch_size=32, 
    num_workers=0)
data_module.setup()

Global seed set to 21
100%|██████████| 42/42 [00:00<00:00, 552.42it/s]


In [7]:
import pandas as pd
df = pd.read_csv('data/3md3070-dlmi/train.csv')
df.head()

,id,label,gender,dob,lymph_count,tiles
0,P154,1,M,6/20/1945,11.97,data/3md3070-dlmi/train/P154/000012.jpg
1,P154,1,M,6/20/1945,11.97,data/3md3070-dlmi/train/P154/000027.jpg
2,P154,1,M,6/20/1945,11.97,data/3md3070-dlmi/train/P154/000035.jpg
3,P154,1,M,6/20/1945,11.97,data/3md3070-dlmi/train/P154/000086.jpg
4,P154,1,M,6/20/1945,11.97,data/3md3070-dlmi/train/P154/000065.jpg


In [8]:
os.makedirs('checkpoints/exp1/')

In [9]:
model = se_resnet50()
model.last_linear = nn.Linear(2048, 1)
clf = MILModel(model, topk=10, aggregation='mean')
trainer = Trainer(
  reload_dataloaders_every_epoch=True, 
  check_val_every_n_epoch=1,
  weights_save_path='checkpoints/exp1',
  gpus=1,
  callbacks=[ProgressBar()])

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth" to /root/.cache/torch/hub/checkpoints/se_resnet50-ce0d4300.pth


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [10]:
def run_training(clf, trainer, data_module):
  assert trainer.reload_dataloaders_every_epoch
  # print('Setting trainer.max_epochs = 1.')
  trainer.max_epochs = 1
  data_module.inference_dataset_reference = data_module.train_dataset
  inference_metrics = trainer.test(clf, datamodule=data_module)[0]
  data_module.train_dataset_reference = data_module.train_dataset
  data_module.validation_dataset_reference = data_module.val_dataset
  trainer.fit(clf, datamodule=data_module)
  training_metrics = clf.training_metrics
  validation_metrics = clf.validation_metrics
  return inference_metrics, training_metrics, validation_metrics

In [11]:
all_training_metrics = []
all_validation_metrics = []
for epoch in range(20):
  inference_metrics, training_metrics, validation_metrics = run_training(clf, trainer, data_module)
  all_training_metrics.append(training_metrics)
  all_validation_metrics.append(validation_metrics)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The testing_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)

  | Name  | Type    | Params
----------------------------------
0 | model | SENet   | 26.0 M
1 | loss  | BCELoss | 0     
----------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0.692307710647583,
 'auc': 0.7469444274902344,
 'precision': 0.692307710647583,
 'recall': 0.692307710647583}
--------------------------------------------------------------------------------



  | Name  | Type    | Params
----------------------------------
0 | model | SENet   | 26.0 M
1 | loss  | BCELoss | 0     
----------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0.5230769515037537,
 'auc': 0.6838889122009277,
 'precision': 0.5230769515037537,
 'recall': 0.5230769515037537}
--------------------------------------------------------------------------------



  | Name  | Type    | Params
----------------------------------
0 | model | SENet   | 26.0 M
1 | loss  | BCELoss | 0     
----------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params


...


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0.7615384459495544,
 'auc': 0.9827777147293091,
 'precision': 0.7615384459495544,
 'recall': 0.7615384459495544}
--------------------------------------------------------------------------------



  | Name  | Type    | Params
----------------------------------
0 | model | SENet   | 26.0 M
1 | loss  | BCELoss | 0     
----------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0.8769230842590332,
 'auc': 0.9491665959358215,
 'precision': 0.8769230842590332,
 'recall': 0.8769230842590332}
--------------------------------------------------------------------------------



  | Name  | Type    | Params
----------------------------------
0 | model | SENet   | 26.0 M
1 | loss  | BCELoss | 0     
----------------------------------
26.0 M    Trainable params
0         Non-trainable params
26.0 M    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0.692307710647583,
 'auc': 0.887222170829773,
 'precision': 0.692307710647583,
 'recall': 0.692307710647583}
--------------------------------------------------------------------------------


In [10]:
# all_validation_metrics

test

In [12]:
from torch.utils.data import DataLoader
from data.dataset import MILImageDataset

test_df = pd.read_csv('data/3md3070-dlmi/test.csv')
test_dataset = MILImageDataset(test_df, training=False)
data_module.inference_dataset_reference = test_dataset
data_module.batch_size = 1
clf.compute_test_metrics = False

In [13]:
test_metrics = trainer.test(clf, datamodule=data_module)[0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'acc': 0,
 'auc': 0.887222170829773,
 'precision': 0.692307710647583,
 'recall': 0.692307710647583}
--------------------------------------------------------------------------------


In [15]:
preds_df = pd.read_csv('inference.csv')
submission_df = preds_df[['id', 'preds']].rename(columns={"id": "Id", "preds": "Predicted"})
submission_df.head()

,Id,Predicted
0,P108,0
1,P114,1
2,P119,0
3,P120,1
4,P132,1


In [17]:
submission_df.to_csv('submission_k=10_mean.csv', index=False)